In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
from concurrent.futures import ThreadPoolExecutor

In [3]:
fake = faker.Faker(locale='en')

In [4]:
apple_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'countrycode': 'USA',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'origin': 'http://jobs.apple.com',
    'Host': 'jobs.apple.com',
    'referer': 'http://jobs.apple.com/en-us/search'
}

In [5]:
MAIN_URL = "http://jobs.apple.com"
HTML_URL = "http://jobs.apple.com/en-us/search"
CSRF_TOKEN = "http://jobs.apple.com/api/csrfToken"
API_URL = "http://jobs.apple.com/api/role/search"

## Через парсинг страниц
Сначала собираем ссылки и начальную инфу для детального парсинга каждой ваки

In [6]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [7]:
jobs_list = []
null_counter = 0
for page in tqdm(range(0, 1000)):
    search_params = {
        'page': page
    }
    html_result = requests.get(HTML_URL, headers=apple_headers, params=search_params)
    soup = BeautifulSoup(html_result.text)

    results_table = soup.find('div', attrs={'class': 'results__table'})
    if results_table:
        null_counter = 0

        jobs = results_table.find('table').find_all('tbody')
        for job_block in jobs:
            job_info = job_block.findNext('td')

            name = job_info.find('a').text
            url = job_info.find('a')['href']
            internal_id = job_info.find('a')['id'].split('-')[-1]
            publish_date = parser.parse(job_info.find('span').findNext().text)

            jobs_list.append((url, internal_id, name, publish_date))

        time.sleep(0.2) # sleep for 200 ms
    else:
        null_counter += 1
        if null_counter == 3:
            break
        else:
            continue

 41%|████      | 409/1000 [18:51<27:14,  2.77s/it]


In [8]:
len(jobs_list)

8126

In [9]:
jobs_list[100]

('/en-us/details/200125389/cn-business-pro?team=APPST',
 '200125389',
 'CN-Business Pro',
 datetime.datetime(2022, 8, 9, 0, 0))

#### Детальный парсинг страниц вакансий

In [10]:
def get_vacancy(params):
    url, internal_id, name, publish_date = params

    try:
        job_website = requests.get(MAIN_URL + url, headers=apple_headers)
        soup = BeautifulSoup(job_website.text)
    except:
        print(f'Error while requesting {MAIN_URL + url}')
        return None

    try:
        description = clear_string(soup.find('div', id='jd-job-summary').span.text)
        qualifications = clear_string(soup.find('div', id='jd-key-qualifications').text)
        responsibilities = clear_string(soup.find('div', id='jd-description').text)
    except:
        print(f'Error while parsing {job_website.url}')
        return None

    job_dict = {
        'title': name,
        'internal_id' : internal_id,
        'url': MAIN_URL + url,
        'description': description,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'Apple',
        'publish_date': publish_date
    }

    return job_dict

In [11]:
with ThreadPoolExecutor(max_workers=10) as executor:
    job_dicts = list(tqdm(executor.map(get_vacancy, jobs_list), total=len(jobs_list)))

  8%|▊         | 687/8126 [03:25<28:45,  4.31it/s]  

Error while parsing https://jobs-prz.apple.com/en-us/details/200409020/us-manager?team=APPST


  9%|▊         | 692/8126 [03:27<36:52,  3.36it/s]

Error while parsing https://jobs-prz.apple.com/en-us/details/200409022/us-manager?team=APPST
Error while parsing https://jobs-prz.apple.com/en-us/details/200400490/administrative-assistant-ai-ml?team=CORSV


 13%|█▎        | 1046/8126 [05:07<32:31,  3.63it/s] 

...

 69%|██████▉   | 5628/8126 [28:52<17:52,  2.33it/s]

Error while parsing https://jobs-prz.apple.com/en-us/details/200366854/administrative-assistant?team=CORSV


 93%|█████████▎| 7592/8126 [38:41<02:24,  3.71it/s]

Error while parsing https://jobs-prz.apple.com/en-us/details/200271917/recruiter-internet-software-services-contract?team=CORSV


100%|██████████| 8126/8126 [41:17<00:00,  3.28it/s]


In [12]:
snapshot = pd.DataFrame(filter(lambda x: x is not None, job_dicts))
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
1937,"Data Scientist - Wallet, Payments & Commerce -...",200400995,http://jobs.apple.com/en-us/details/200400995/...,Apple is a place where extraordinary people ga...,Take deep dives in large-scale data to identif...,"Working knowledge of relational databases, inc...",Apple,2022-07-19
2315,Supply Demand Management,200371437,http://jobs.apple.com/en-us/details/200371437/...,COMPANY OVERVIEW: \n Apple ignited the persona...,•\tBe the supply and demand representative in ...,"BA/BS required; MBA preferred, or equivalent w...",Apple,2022-07-13
6980,Performance Engineer,200323237,http://jobs.apple.com/en-us/details/200323237/...,The Technology Development Group is working on...,The Technology Development Group is looking fo...,In-depth knowledge of tools and techniques for...,Apple,2021-12-10
5746,Machine Learning Scientist - Health Sensing,200365146,http://jobs.apple.com/en-us/details/200365146/...,"""I believe, if you zoom out into the future, a...",This is a key position on a team supporting R&...,5+ years of experience in machine learning alg...,Apple,2022-04-06
4002,"Software Engineer, Trust and Safety",200360404,http://jobs.apple.com/en-us/details/200360404/...,"At Apple, our Trust & Safety team is responsib...",This is a hands-on software development role. ...,8 - 10 years experience working on large-scale...,Apple,2022-06-06


In [13]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8105 entries, 0 to 8104
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             8105 non-null   object        
 1   internal_id       8105 non-null   object        
 2   url               8105 non-null   object        
 3   description       8105 non-null   object        
 4   responsibilities  8105 non-null   object        
 5   qualifications    8105 non-null   object        
 6   company           8105 non-null   object        
 7   publish_date      8105 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 506.7+ KB


In [14]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [16]:
snapshot.to_csv(f'../data/apple/{current_date}.csv')
snapshot.to_excel(f'../data/apple/{current_date}.xlsx', engine='xlsxwriter')

#### Проверка на единичной вакансии

In [202]:
job_website = requests.get(MAIN_URL + jobs_list[0][0], headers=apple_headers)
job_website

<Response [200]>

In [204]:
soup = BeautifulSoup(job_website.text)

In [214]:
description = soup.find('div', id='jd-job-summary').span.text
clear_string(description)

'Do you love how it feels to help others? After customers purchase our products, you’re the one who helps them get more out of their new Apple technology. Your day in the Apple Store is filled with a range of focused support and service tasks. Whether you’re helping customers get started with the Mac or finding answers to their questions about other Apple devices, you’re ready to share knowledge and provide exceptional assistance. You gain satisfaction from bringing resolution and insight to each customer, elevating his or her relationship with Apple to the next level.\n \n Both full-time and part-time jobs are available.'

In [212]:
soup.find('div', id='jd-key-qualifications').text

'Ability to assess customers’ support needs when they arrive, then provide solutions or refer them to other team membersFlexibility to regularly rotate through different technical specialities and skill setsAbility to thrive on change as products evolve'

In [213]:
soup.find('div', id='jd-description').text

'As a Technical Specialist, you help new owners get started and current ones get quick, efficient support — developing strong, positive relationships with Apple. When a customer needs assistance, you quickly assess their situation. Sometimes you take care of customers with advice or a solution on the spot, using your knowledge of current Apple technology to help with iPod, iPhone and iPad devices. At other times, you refer customers to support team members who get them up and running again. You even provide personal training for new customers, helping them acquire the basic skills they need to get started on photo, video and music projects. The entire store team benefits from your commitment to providing the best care for customers. By helping Apple maintain strong relationships with customers, you are instrumental to our success.\n\nDiscover even more benefits of doing what you love. Apple’s most important resource, our soul, is our people. Apple benefits help further the well-being o

## Через API бэкенда
ВОозникают проблемы, поэтому выбран обычный парсинг

In [14]:
session = requests.Session()
session.headers = apple_headers

In [84]:
result = session.get(CSRF_TOKEN)
result

<Response [200]>

In [85]:
session.headers['X-Apple-CSRF-Token'] = result.headers['X-Apple-CSRF-Token']

In [86]:
session.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; PPC Mac OS X 10_9_7) AppleWebKit/535.0 (KHTML, like Gecko) Chrome/55.0.854.0 Safari/535.0',
 'accept-language': 'en-US,en;q=0.9',
 'pragma': 'np-cache',
 'countrycode': 'USA',
 'sec-fetch-mode': 'cors',
 'sec-fetch-site': 'same-origin',
 'origin': 'https://jobs.apple.com',
 'Host': 'jobs.apple.com',
 'referer': 'https://jobs.apple.com/en-us/search',
 'X-Apple-CSRF-Token': '2da62a5db304cb69f52f5ca8ddec1f1eb8aff5bca75024853a4959f5ba64c62a'}

In [89]:
search_payload = {
    'page': 1,
    'query': '',
    'filters': {"range":{"standardWeeklyHours":{"start":None,"end":None}}},
    'locals': 'en-us',
    'sort': 'relevance',
}

In [90]:
result = session.post(API_URL, data=search_payload, allow_redirects=True)
result

<Response [403]>

In [ ]:
result.text

In [ ]:
print(result.text)